In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
dataset_names = ['Clientes', 'Compra', 'Gasto', 'Localidades','Proveedores', 'Sucursales', 'Venta']

In [5]:
#Levantamos los datasets en crudo (archivos .csv)
df_clientes     =pd.read_csv('../Datasets/Clientes.csv'   , encoding='latin-1', sep=';')
df_compras      =pd.read_csv('../Datasets/Compra.csv'     , encoding='latin-1', sep=',')
df_gastos       =pd.read_csv('../Datasets/Gasto.csv'      , encoding='latin-1', sep=',')
df_localidades  =pd.read_csv('../Datasets/Localidades.csv', encoding='latin-1', sep=',')
df_proveedores  =pd.read_csv('../Datasets/Proveedores.csv', encoding='latin-1', sep=',')
df_sucursales   =pd.read_csv('../Datasets/Sucursales.csv' , encoding='latin-1', sep=';')
df_ventas       =pd.read_csv('../Datasets/Venta.csv'      , encoding='latin-1', sep=',')

In [10]:
df_clientes.head(1)

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,"-58,81850307","-34,30997088",NaN


In [9]:
print(df_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2672 non-null   int64  
 1   Provincia          2647 non-null   object 
 2   Nombre_y_Apellido  2648 non-null   object 
 3   Domicilio          2630 non-null   object 
 4   Telefono           2637 non-null   object 
 5   Edad               2672 non-null   int64  
 6   Localidad          2647 non-null   object 
 7   X                  2616 non-null   object 
 8   Y                  2618 non-null   object 
 9   col10              0 non-null      float64
dtypes: float64(1), int64(2), object(7)
memory usage: 208.9+ KB
None


Renombrando columnas

In [ ]:
#Clientes
columnas_nuevas = ['provincia', 'nom_y_ape', 'domicilio', 'tel', 'edad', 'localidad', 'lat', 'long', 'col10']
df_clientes.columns = columnas_nuevas

#Compras
columnas_nuevas = ['fecha', 'año', 'mes', 'periodo', 'idProducto','cantidad', 'precio', 'idProveedor']
df_compras.columns = columnas_nuevas

#Gastos
columnas_nuevas = ['idSucursal', 'idTipoGasto', 'fecha', 'monto']
df_gastos.columns = columnas_nuevas

#Localidades
columnas_nuevas = ['categoria','lat','lon','idDepartamento','nombreDepartamento','fuente','idLocalidadCensal','nombreLocalidadCensal','idMunicipio','nombreMunicipio','nombreLocalidad','idProvincia','nombreProvincia']
df_localidades.columns = columnas_nuevas

#Proveedores
columnas_nuevas = ['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'departamento']
df_proveedores.columns = columnas_nuevas

#Sucursales
columnas_nuevas = ['sucursal', 'direccion', 'localidad', 'provincia', 'lat', 'long']
df_sucursales.columns = columnas_nuevas

#Ventas
columnas_nuevas = ['fechaVenta','fechaEntrega','idCanal','idCliente','idSucursal','idEmpleado','idProducto','precio','cantidad']
df_ventas.columns = columnas_nuevas

Renombrando indices

In [ ]:
df_clientes.index.name = 'idCliente'
df_compras.index.name = 'idCompra'
df_gastos.index.name = 'idGasto'
df_localidades.index.name = 'idLocalidad'
df_proveedores.index.name = 'idProveedor'
df_sucursales.index.name = 'idSucursal'
df_ventas.index.name = 'idVenta'